In [1]:
from ibm_watson_machine_learning import APIClient

In [2]:
wml_credential = {
    'url': 'https://eu-de.ml.cloud.ibm.com',
    'apikey': 'Ndj3UnNat-ZW_rOzehbxfzaMRt5DfuLeISJfNW_vuebX'
}

In [3]:
client = APIClient(wml_credential)

In [4]:
client

In [5]:
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  --------------  ------------------------
ID                                    NAME            CREATED
4ecc4a52-91f7-4d31-a885-6851624e76ec  Nalaiya thiran  2022-11-04T06:05:27.721Z
------------------------------------  --------------  ------------------------


In [6]:
space_id = "4ecc4a52-91f7-4d31-a885-6851624e76ec"
client.set.default_space(space_id)

'SUCCESS'

In [7]:
software_space_id = client.software_specifications.get_id_by_name("tensorflow_rt22.1-py3.9")
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a1

In [8]:
software_space_id

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [9]:
model_details = client.repository.store_model(
    model='model.tgz',
    meta_props={
    client.repository.ModelMetaNames.NAME:"CVD predictor",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_id }
)

In [10]:
client.repository.list()

------------------------------------  -------------  ------------------------  --------------  -----
GUID                                  NAME           CREATED                   FRAMEWORK       TYPE
b800b511-d515-47da-8bd5-c06b927c379e  CVD predictor  2022-11-04T06:34:57.002Z  tensorflow_2.7  model
------------------------------------  -------------  ------------------------  --------------  -----


In [12]:
client.hardware_specifications.list()

----------------------  ------------------------------------  --------------------------------------------------------------------------------------------------------------------------------------
NAME                    ID                                    DESCRIPTION
ML                      5a1f0e64-e420-55ce-bd7a-f6d00bb942cf  A hardware specification providing 4 CPU cores and 32 GiB of memory.
L                       a6c4923b-b8e4-444c-9f43-8a7ec3020110  A hardware specification providing 8 CPU cores and 32 GiB of memory.
Default Spark           ac59d20a-9c7c-4504-a853-788ef35969da  A hardware specification for Spark with 1 CPU and 4 GiB of memory for master and worker nodes, with 2 workers.
XXS                     b128f957-581d-46d0-95b6-8af5cd5be580  A hardware specification providing one CPU core and 2 GiB of memory.
M-Spark                 b2232f7a-bfad-4822-9bce-6ba1af49217a  A hardware specification for Spark service with 2 CPU and 8 GiB of memory for master and 2 CPU and 8 

In [14]:
hardware_spec_id = "b128f957-581d-46d0-95b6-8af5cd5be580"
model_id = "b800b511-d515-47da-8bd5-c06b927c379e"
dep_details = client.deployments.create(artifact_uid=model_id,meta_props={
     client.deployments.ConfigurationMetaNames.NAME:"Nalaiya_thiran_deployment",
     client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { "id": hardware_spec_id},
     client.deployments.ConfigurationMetaNames.ONLINE:{}})



#######################################################################################

Synchronous deployment creation for uid: 'b800b511-d515-47da-8bd5-c06b927c379e' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
..
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='b48cd709-a43a-45af-8f80-8535affa8849'
------------------------------------------------------------------------------------------------




In [15]:
from tensorflow.keras.preprocessing import image
import numpy as np

In [16]:
img = np.expand_dims(image.img_to_array(
            image.load_img("ecg1.png", target_size=(64, 64), color_mode="grayscale")), axis=0)

In [17]:
scoring_payload = {
 client.deployments.ScoringMetaNames.INPUT_DATA: [{
        "values": img
     }]
 }

In [18]:
dep_id = "b48cd709-a43a-45af-8f80-8535affa8849"

In [19]:
client.deployments.score(deployment_id=dep_id,meta_props=scoring_payload)

{'predictions': [{'id': 'dense_1',
   'fields': ['prediction', 'prediction_classes', 'probability'],
   'values': [[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
     0,
     [1.0, 0.0, 0.0, 0.0, 0.0, 0.0]]]}]}